In [4]:
import pandas as pd
df = pd.read_csv("archive/spam.csv", encoding='latin-1')
print(df.shape)
print(df.head())

(5572, 5)
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [5]:
df = df[['v1', 'v2']]
df.columns = ['label', 'message']
print(df.shape)
print(df.head())

(5572, 2)
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [6]:
df = df.iloc[:,:2]
df.columns = ['label', 'message']
print(df.shape)
print(df.head())

(5572, 2)
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [7]:
!pip install nltk

In [10]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\PALAK
[nltk_data]     NAGPAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def clean_text(text):
    text = str(text).lower()  # convert to lowercase
    text = re.sub(r"https?://\S+|www\.\S+", " <URL> ", text)  # replace URLs
    text = re.sub(r"\d+", " <NUM> ", text)  # replace numbers
    text = re.sub(r"[^\w\s<>]", " ", text)  # remove punctuation
    tokens = text.split()  # split into words
    tokens = [t for t in tokens if t not in stopwords.words('english')]  # remove stopwords
    return " ".join(tokens)
    

In [14]:
df['message_clean'] = df['message'].apply(clean_text)
print(df[['message', 'message_clean']].head())

                                             message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                       message_clean  
0  go jurong point crazy available bugis n great ...  
1                            ok lar joking wif u oni  
2  free entry <NUM> wkly comp win fa cup final tk...  
3                u dun say early hor u c already say  
4             nah think goes usf lives around though  


In [15]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score
import joblib

# Convert label to numeric: 'ham' → 0, 'spam' → 1
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

X = df['message_clean']
y = df['label_num']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.85, min_df=2)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the Naive Bayes model
model = MultinomialNB()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print("F1 Score:", f1_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

joblib.dump(model, 'model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')
print("\nModel and vectorizer saved successfully!")

F1 Score: 0.9084249084249084

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       966
           1       1.00      0.83      0.91       149

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115


Model and vectorizer saved successfully!


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\PALAK
[nltk_data]     NAGPAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import re
import joblib
import nltk
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

def clean_text(s):
    s = str(s).lower()
    s = re.sub(r"https?://\S+|www\.\S+", " <URL> ", s)
    s = re.sub(r"\d+", " <NUM> ", s)
    s = re.sub(r"[^\w\s<>]", " ", s)
    tokens = s.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    return " ".join(tokens)

model = joblib.load("model.joblib")
vectorizer = joblib.load("vectorizer.joblib")

sample_sms = "Congratulations! You have won a free ticket to Bahamas. Call now!"

cleaned_sms = clean_text(sample_sms)

vectorized_sms = vectorizer.transform([cleaned_sms])

prediction = model.predict(vectorized_sms)[0]
probability = model.predict_proba(vectorized_sms)[0][1]

label = "Spam" if prediction == 1 else "Legitimate"
print(f"Predicted label: {label}")
print(f"Probability of being spam: {probability:.2f}")
print(f"Cleaned message: {cleaned_sms}")

Predicted label: Legitimate
Probability of being spam: 0.29
Cleaned message: congratulations free ticket bahamas call
